In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import pandas as pd
import numpy as np
from tqdm import tqdm
import time
import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize
import gc
tqdm.pandas()
gc.collect()

C:\Users\ricky\anaconda3\lib\site-packages\tqdm\std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


15

# LOAD

In [2]:
%%time
df = pd.read_feather('../data/2021玉山人工智慧公開挑戰賽冬季賽訓練資料集.feather')
df = df.loc[df.dt>=12,:] #取最近一年即可
print(df.shape,df.isnull().sum().sum())
df.head()

(19786604, 53) 1509444
Wall time: 58.3 s


,dt,chid,shop_tag,txn_cnt,txn_amt,domestic_offline_cnt,domestic_online_cnt,overseas_offline_cnt,overseas_online_cnt,domestic_offline_amt_pct,...,masts,educd,trdtp,naty,poscd,cuorg,slam,gender_code,age,primary_card
13188928,12.0,10115966.0,15,1.0,9678.450445,1.0,0.0,0.0,0.0,1.00,...,1.0,4.0,15.0,1.0,1.0,30.0,133000.939458,0.0,5.0,1.0
13188938,12.0,10115966.0,18,1.0,9624.614033,1.0,0.0,0.0,0.0,1.00,...,1.0,4.0,15.0,1.0,1.0,30.0,133000.939458,0.0,5.0,1.0
13188944,12.0,10115966.0,25,1.0,2338.181448,1.0,0.0,0.0,0.0,1.00,...,1.0,4.0,15.0,1.0,1.0,30.0,133000.939458,0.0,5.0,1.0
13188946,12.0,10115966.0,26,2.0,16627.150110,2.0,0.0,0.0,0.0,1.00,...,1.0,4.0,15.0,1.0,1.0,30.0,133000.939458,0.0,5.0,1.0
13188957,12.0,10115966.0,37,13.0,18485.113446,3.0,10.0,0.0,0.0,0.27,...,1.0,4.0,15.0,1.0,1.0,30.0,133000.939458,0.0,5.0,1.0


# 處理缺失值

In [3]:
%%time
# Function to calculate missing values by column# Funct 
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns
miss_table = missing_values_table(df)
miss_table

Your selected dataframe has 53 columns.
There are 9 columns that have missing values.
Wall time: 6.2 s


,Missing Values,% of Total Values
slam,1023527,5.2
gender_code,242749,1.2
age,242749,1.2
trdtp,365,0.0
educd,14,0.0
masts,10,0.0
naty,10,0.0
poscd,10,0.0
cuorg,10,0.0


In [4]:
mean_fill_col = ['slam'] # 數值型用中位數補值(避免離群值拉高平均值)
cat_fill_col = ['gender_code','age','trdtp','educd','masts','naty','poscd','cuorg'] # 類別型用一個-999補值
for i in tqdm(mean_fill_col):
    df[i] = df[i].fillna(df[i].median())
for i in tqdm(cat_fill_col):
    df[i] = df[i].fillna(-999)
miss_table = missing_values_table(df)
miss_table

100%|██████████| 8/8 [00:01<00:00,  6.55it/s]


Your selected dataframe has 53 columns.
There are 0 columns that have missing values.


,Missing Values,% of Total Values


In [6]:
df.isnull().sum().sum()

0

In [10]:
chid = df.head(1)['chid'].values[0]
chid

10115966.0

# groupby('shop_tag')觀察txn_amt的sum並做排序

In [16]:
df.loc[df.chid==chid,['shop_tag','txn_amt']].groupby('shop_tag').agg('sum').sort_values(by='txn_amt',ascending=False)

,txn_amt
shop_tag,
2,232533.602882
37,220452.379787
18,199466.732709
25,149987.199235
15,85987.255801
9,80153.137127
26,73621.598094
21,48616.442024
19,43486.774335


# 該chid之shop_tag的value_counts()

In [18]:
df.loc[df['chid']==chid,'shop_tag'].value_counts().to_frame()

,shop_tag
37,13
9,13
18,12
2,11
25,10
15,8
26,6
other,5
21,4
19,3


In [42]:
官方指認欄位 = ['2','6','10','12','13','15','18','19','21','22','25','26','36','37','39','48']
def chid2answer(chid):
    a = df.loc[df.chid==chid,['shop_tag','txn_amt']].groupby('shop_tag').agg('sum').sort_values(by='txn_amt',ascending=False)
    a['在指認欄位'] = False
    a.loc[list(set(a.index)&set(官方指認欄位)),'在指認欄位'] = True #有交集的部份做個記號
    answer = a[a['在指認欄位']==True].head(3)
    display(answer)
    return answer.index.tolist()

In [44]:
lst = []
for _ in tqdm(range(3)):
    answer = chid2answer(np.random.choice(df['chid'].values))
    lst.append(answer)
lst

  0%|          | 0/3 [00:00<?, ?it/s]

,txn_amt,在指認欄位
shop_tag,,
2,233937.144865,True
15,150782.766175,True
18,116659.077968,True


,txn_amt,在指認欄位
shop_tag,,
10,241000.524631,True
2,176730.747450,True
15,113560.280446,True


,txn_amt,在指認欄位
shop_tag,,
37,154459.962110,True
22,5862.917536,True
21,5510.328752,True


100%|██████████| 3/3 [00:00<00:00, 35.06it/s]


[['2', '15', '18'], ['10', '2', '15'], ['37', '22', '21']]